In [1]:
import os
os.chdir('..')
os.getcwd()

'/Users/leowyy/Desktop/fyp-graph-clustering'

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import pickle 
import numpy as np
import pandas as pd
import torch
import os
from timeit import default_timer as timer
import networkx as nx
import scipy.sparse as sp

from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import pairwise_distances

In [3]:
from core.SimpleNet import SimpleNet
from core.GraphConvNet import GraphConvNet
from core.EmbeddingDataSet import EmbeddingDataSet
from core.GraphDataBlock import GraphDataBlock
from util.plot_graph_embedding import plot_graph_embedding
from util.evaluation_metrics import evaluate_viz_metrics
from util.network_utils import get_net_projection
from util.graph_utils import get_shortest_path_matrix, neighbor_sampling

cuda not available
cuda not available


In [4]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [5]:
if torch.cuda.is_available():
    print('cuda available')
    device = 'cuda'
else:
    print('cuda not available')
    device = 'cpu'

cuda not available


In [7]:
dataset_name = 'citeseer_test'
parent_dir = os.path.abspath('..')
input_dir = os.path.join(parent_dir, 'data')
dataset = EmbeddingDataSet(dataset_name, input_dir, train=True)
dataset.create_all_data(n_batches=1, shuffle=False)
dataset.summarise()

Data blocks of length:  [3312]
Time to create all data (s) = 0.1146
Name of dataset = citeseer_test
Input dimension = 3703
Number of training samples = 3312
Training labels = True


In [8]:
dataset = EmbeddingDataSet(dataset_name, input_dir, train=True)
dataset.create_all_data(n_batches=1, shuffle=False)

Data blocks of length:  [3312]
Time to create all data (s) = 0.0458


# Visualization

In [9]:
net_parameters = {}
net_parameters['n_components'] = 2
net_parameters['D'] = dataset.input_dim # input dimension
net_parameters['L'] = 2 # number of hidden layers

In [24]:
# Graph net
net_parameters['n_components'] = 2
net_parameters['H'] = 128 # number of hidden units

net_1 = GraphConvNet(net_parameters)
if torch.cuda.is_available(): net_1.cuda()
root = 'results/citeseer_full_6/'
filename = root + 'graph_net_360.pkl'
checkpoint = torch.load(filename, map_location=device)
net_1.load_state_dict(checkpoint['state_dict'])

In [25]:
y_pred_1 = get_net_projection(net_1, dataset)

Data blocks of length:  [3312]
Time to create all data (s) = 0.0526


In [26]:
plot_graph_embedding(y_pred_1, dataset.labels, dataset.adj_matrix, line_alpha=0.1)

In [27]:
_ = evaluate_viz_metrics(y_pred_1, dataset)

One NN accuracy = 0.6645
Avg graph distance = 0.0537
Avg feature distance = 1.4053
Total distance = 1.4590


In [28]:
dataset.inputs.shape

(3312, 3703)

In [32]:
main_dir = os.path.join('results', dataset_name)
main_dir

'results/citeseer_test'

In [33]:
path_matrix = np.load(os.path.join(input_dir, dataset_name) + '/path_matrix.npy')

In [34]:
embedder = TSNE(n_components=2, metric='precomputed', perplexity=30, verbose=1)
y_pred_tsne = embedder.fit_transform(path_matrix)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 3312 samples in 0.006s...
[t-SNE] Computed neighbors for 3312 samples in 0.124s...
[t-SNE] Computed conditional probabilities for sample 1000 / 3312
[t-SNE] Computed conditional probabilities for sample 2000 / 3312
[t-SNE] Computed conditional probabilities for sample 3000 / 3312
[t-SNE] Computed conditional probabilities for sample 3312 / 3312
[t-SNE] Mean sigma: 0.957222
[t-SNE] KL divergence after 250 iterations with early exaggeration: 74.125671
[t-SNE] KL divergence after 1000 iterations: 1.092515


In [35]:
plot_graph_embedding(y_pred_tsne, dataset.labels, dataset.adj_matrix, line_alpha=0.1)

In [31]:
_ = evaluate_viz_metrics(y_pred_tsne, dataset)

One NN accuracy = 0.6673
Avg graph distance = 1.1212
Avg feature distance = 0.8855
Total distance = 2.0067
